# BiteMe | Train

This notebook includes the most important part of the project - the modelling. The notebook tests methodologies for training, and in it the chosen algorithm is decided. Validation also occurs before final testing, which is conducted in the test notebook. This stage is highly iterative, so all model artefacts, logs and configurations are recorded and saved to disk automatically. This initial setup of what will eventually become MLOps for the final product will be really useful, and helps keep track of what is successful and what isn't.

Models to try:
 - resnet50v2
 - resnet101v2
 - resnet152v2
 - vgg19
 - densenet169
 - densenet121
 - densenet201
 - inceptionv3
 - inception_resnetv2
 - resnext50
 - resnext101
 - xception
 - efficientnet_b0
 - efficientnet_b1
 - efficientnet_b2
 - efficientnet_b3
 - efficientnet_b4
 - efficientnet_b5

Initial model work is done by using simple, typical image recognition models (CNN architectures) to see how effective these models can be for the problem. Although I don't 


In [1]:
# Basic imports
import pandas as pd
import numpy as np
import os
import sys

# Data visualisation
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn

# Modelling imports
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score

# Image processing
import cv2
import albumentations as A
import imgaug as ia
import imgaug.augmenters as iaa

import torch

# Local imports
sys.path.append("..")
from helpers import read_images, augs, get_augs
from constants import *

plt.rcParams["figure.figsize"] = (14, 8)

np.random.seed(SEED)
ia.seed(SEED)

In [2]:
# Define directories
base_dir_path = "../"

data_dir_path = os.path.join(base_dir_path, "data")
data_preprocessed_dir_path = os.path.join(data_dir_path, "preprocessed")
data_preprocessed_train_dir_path = os.path.join(data_dir_path, "preprocessed/train")

data_dir = os.listdir(data_dir_path)
data_preprocessed_dir = os.listdir(data_preprocessed_dir_path)
data_preprocessed_train_dir = os.listdir(data_preprocessed_train_dir_path)

metadata_preprocessed_path = os.path.join(data_preprocessed_dir_path, "metadata.csv")
metadata = pd.read_csv(metadata_preprocessed_path)
# Subset to train only
metadata = metadata.loc[metadata.split == "train"]

metadata.head()

,img_name,img_path,label,split
0,7059b14d2aa03ed6c4de11afa32591995181d31c.jpg,../data/cleaned/none/7059b14d2aa03ed6c4de11afa...,none,train
1,ea1b100b581fcdb7ddfae52cc62347a99e304ba4.jpg,../data/cleaned/none/ea1b100b581fcdb7ddfae52cc...,none,train
3,6eac051b9c45ff6821ec8675216f371711b7cea9.jpg,../data/cleaned/none/6eac051b9c45ff6821ec86752...,none,train
4,fc72767f8520df9b2b83941077dc0ee013eb9399.jpg,../data/cleaned/none/fc72767f8520df9b2b8394107...,none,train
5,cf812984268e2aec9a167d3ebe1026f610dd862b.jpg,../data/cleaned/none/cf812984268e2aec9a167d3eb...,none,train


In [3]:
# Read in train images
X_train = read_images(
    data_dir_path=data_preprocessed_train_dir_path, 
    rows=ROWS, 
    cols=COLS, 
    channels=CHANNELS, 
    write_images=False, 
    output_data_dir_path=None,
    verbose=VERBOSE
)

# Get labels
y_train = np.array(metadata["label"])

Reading images from: ../data/preprocessed/train
Rows set to 512
Columns set to 512
Channels set to 3
Writing images is set to: False
Reading images...


100%|██████████████████████████████████████████████████████████████████████████████| 23/23 [00:00<00:00, 29.17it/s]


Image reading complete.
Image array shape: (192, 512, 512, 3)


## Set Parameters

In [4]:
# Choose augmentations to use in preprocessing
# For full list see helpers.py
augs_to_select = [
    "Fliplr", 
    "Flipud", 
    "Cutout"
]
# Subset augs based on those selected
augs = dict((aug_name, augs[aug_name]) for aug_name in augs_to_select)

# Modelling constants - add this to constants.py when needed
MODEL_NAME = "resnet50v2"
EPOCHS = 6



# Create dictionary of configurations used in modelling
# this will be updated as modelling progresses if necessary, for logging
conf = {
    "device": "cuda" if torch.cuda.device_count() > 0 else "cpu",
    "device_name": torch.cuda.get_device_name(0),
    "n_workers": torch.cuda.device_count(),
    "rows": ROWS,
    "cols": COLS,
    "channels": CHANNELS,
    "seed": SEED,
    "num_classes": list(metadata["label"].unique()),
    "classes": np.unique(y_train, return_counts=True)[0],
    "class_counts": np.unique(y_train, return_counts=True)[1],
    "test_size": TEST_SIZE,
    "num_train_sample": y_train.shape[0],
    "val_size": 0.15,
    "num_augs": len(augs),
    "augs": augs,
    "model_name": MODEL_NAME,
    "batch_size": 16,
    "epochs": EPOCHS,
    "lr": 1e-5,
    "optimizer": "AdamW",
    "n_splits": N_SPLITS
}

/home/edwardsims/miniconda3/envs/biteme/lib/python3.10/site-packages/torch/cuda/__init__.py:146: UserWarning: 
NVIDIA GeForce RTX 3090 with CUDA capability sm_86 is not compatible with the current PyTorch installation.
The current PyTorch install supports CUDA capabilities sm_37 sm_50 sm_60 sm_61 sm_70 sm_75 compute_37.
If you want to use the NVIDIA GeForce RTX 3090 GPU with PyTorch, please check the instructions at https://pytorch.org/get-started/locally/

  warnings.warn(incompatible_device_warn.format(device_name, capability, " ".join(arch_list), device_name))


In [4]:
torch.version.cuda

In [5]:
torch.cuda.get_device_name(0)

AssertionError: Torch not compiled with CUDA enabled

In [6]:
torch.cuda.device_count()

0

In [7]:
torch.cuda.is_available()


False

In [ ]:
torch.rand(5, 3)


In [ ]:
# Split cross validation idx
# Subset images and labels for cross validation
# Create image augmentations and additional labels
# Read in pretrained weights
# Any additional layers
# Create model instance
# Create error metric
# Run training
# Make val predictions
# Val error metric
# Create directory for instance
# Save model
# Save log and config 
# Append train/val errors to csv

In [ ]:
skf = StratifiedKFold(n_splits=3)
for train_index, test_index in skf.split(metadata.index, metadata["label"]):
    print(train_index)
    print("-"*40)

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms

In [ ]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 4

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# functions to show an image


def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()


# get some random training images
dataiter = iter(trainloader)
images, labels = dataiter.next()

# show images
imshow(torchvision.utils.make_grid(images))
# print labels
print(' '.join(f'{classes[labels[j]]:5s}' for j in range(batch_size)))


In [1]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()

In [2]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [ ]:
for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0

print('Finished Training')

In [4]:
import torch
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# Assuming that we are on a CUDA machine, this should print a CUDA device:

print(device)

cuda:0


In [5]:
net.to(device)

/home/edwardsims/miniconda3/envs/biteme/lib/python3.10/site-packages/torch/cuda/__init__.py:146: UserWarning: 
NVIDIA GeForce RTX 3090 with CUDA capability sm_86 is not compatible with the current PyTorch installation.
The current PyTorch install supports CUDA capabilities sm_37 sm_50 sm_60 sm_61 sm_70 sm_75 compute_37.
If you want to use the NVIDIA GeForce RTX 3090 GPU with PyTorch, please check the instructions at https://pytorch.org/get-started/locally/

  warnings.warn(incompatible_device_warn.format(device_name, capability, " ".join(arch_list), device_name))


Net(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

In [ ]:
inputs, labels = data[0].to(device), data[1].to(device)